In [42]:
from pymongo import MongoClient
import requests
import json
import pandas as pd
from getpass import getpass
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [43]:
def access_mdb_local_collection (database, collection):

    """ Function that returns a collection inside a DataBase 
    host in my local Server """

    # 1. Connect to my local Server
    client = MongoClient("localhost:27017")
    
    # 2. Access a DataBase:
    db = client[database]

    # 3. Acces a collection inside the given DataBase
    c = db.get_collection(collection)

    # 4. Return collection into a given DataBase host in a given Server
    return c

1. Create a map for each of the posible cities:

In [44]:
locations = {
    "Madrid" : [40.40841191, -3.68760088],
    "Barcelona" : [41.38424664, 2.17634927],
    "Valencia" : [39.47534441, -0.37565717],
    "Sevilla" : [37.38620512, -5.99251368]  
}

In [45]:
mad_map = Map(locations["Madrid"], zoom_start=12, tiles='CartoDB positron')
bcn_map = Map(locations["Barcelona"], zoom_start=12, tiles='CartoDB positron')
vlc_map = Map(locations["Valencia"], zoom_start=12,  tiles='CartoDB positron')
sev_map = Map(locations["Sevilla"], zoom_start=12,  tiles='CartoDB positron')

2. Set the markers specifications

In [46]:
markers = {
    10051: ["stadium" , "orange", "black", "baseball"],
    16033: ["dog_park", "green", "white", "dog"],
    13377: ["vegan_rest", "white", "green", "leaf"],
    10032: ["nigth_club", "black", "white", "champagne-glasses"]
}

### Functions:

In [47]:
def extract_parameters(i):
            
            name = i["name"]
            
            lat = i["geocodes"]["main"]["latitude"]
            lon = i["geocodes"]["main"]["longitude"]
            
            # as elements can have more than one categorie
            #we need to make sure we use the ones that we
            #have consider in markers. We know they all have
            #at least one of them because we extract the 
            #info from the API using categorie filter:
            
            
            cat_list = [x["id"] for x in i["categories"]]
            cat_code = [key for key in markers.keys() if key in cat_list][0]
              
            """
            The same before comprehension list:
            
            for key in markers.keys():
                if key in cat_list
                    cat_code = key
                    
            With the comprehension list, we need to acces the first 
            element -> we know there will be just one.
            """
            params = {"name": name, "cat_code":cat_code, "lat":lat, "lon":lon}
            
            return params

In [48]:
def city_places(city):
    
    "Returs a dataframe with all palces inside a city from our DB"
    
    c = access_mdb_local_collection ("ironhack", "foursquare")
    
    # 1. Set the filter to the objects --> REVIEW option just locality??¿?¿ lo de Valencia - comunidad valencia 
    filt = {"location.locality":city}
    
    # 2. Select the filds that we want to target:
    projection = { "_id":0,"categories":1, "geocodes":1,"name":1}
    
    # 3. Extract data
    datos = list(c.find(filt,projection))
    
    # 4. Generate an empty dictionary with the categories to save:
    register = {"city": [], "place_name":[], "lable":[],"lat":[], "lon":[]}
    
    # 5. Iterate trowh all elements found:
    for i in datos:

        info = extract_parameters(i) # info it's a dictionary 
            # example --> info = {"name": name, "cat_code":cat_code, "lat":lat, "lon":lon}
            
            
        name = info["name"]
        lable =info["cat_code"]
        lat = info["lat"]
        lon =  info["lon"]
        
        register["city"].append(city)
        register["place_name"].append(name)
        register["lable"].append(lable)
        register["lat"].append(lat)
        register["lon"].append(lon)

        df = pd.DataFrame(register)
        
    return df

In [49]:
def icon_set(category_code): # marker of categorye [10051]
    
    icon =  Icon(
            color = markers[category_code][1], #red
            icon_color = markers[category_code][2], #black
            icon = markers[category_code][3], #dog
            prefix="fa" # font-awesome website: fa
                )
            
    return icon

In [50]:
def city_markers(df, to_map):
    
        for index, row in df.iterrows():
            
            #What kind of place it is: (code)
            category_code = df.iloc[index]["lable"]
            
            #Set the icon for that kind of place:
            icon_conf = icon_set(category_code)
            
            #Extract position in witch locates:
            lat = df.iloc[index]["lat"]
            lon = df.iloc[index]["lon"]
            coordenates = [lat, lon]
            
            #Retrieve the category_code name (ej:"stadium")
            # we know the position in the dictionary
            lable = markers[category_code][0]
            
            #Set the marker:
            marker = Marker(coordenates, tooltip=lable ,  icon=icon_conf)
            
            marker.add_to(to_map)
        
        return

In [51]:
def heat_city(df, to_map):
    
    HeatMap(data = df[["lat", "lon"]]).add_to(to_map)
        
    return

### Apply function to or selected cityes:

In [52]:
location = "Valencia"

In [56]:
#map and dataframe
city_map = Map(locations["Valencia"], zoom_start=13,  tiles='CartoDB positron')
df = city_places(location)


city_markers(df, city_map)
city_map.save(f"{location}_markmap.html")

In [57]:
#map and dataframe
city_map =  Map(locations["Valencia"], zoom_start=13,  tiles='CartoDB positron')
df = city_places(location)


heat_city(df, city_map)
city_map.save(f"{location}_heatmap.html")

# MAKE IT BY LAYERS ?¿?:

### Inspect the markers per city:

In [199]:
mark_mad["lable"].value_counts()

lable
nigth_club    53
vegan_rest    52
stadium       45
dog_park      34
Name: count, dtype: int64

In [200]:
mark_vlc["lable"].value_counts()

lable
nigth_club    12
stadium        7
vegan_rest     5
dog_park       3
Name: count, dtype: int64

In [201]:
mark_sev["lable"].value_counts()

lable
nigth_club    51
stadium       20
vegan_rest     7
dog_park       2
Name: count, dtype: int64

### Create the HeatMap

In [202]:
df = mark_sev
df

,lable,lat,lon
0,stadium,37.378758,-5.996984
1,stadium,37.388063,-6.010549
2,stadium,37.384081,-5.970731
3,stadium,37.384046,-5.970596
4,stadium,37.384046,-5.970596
...,...,...,...
75,nigth_club,37.407838,-5.933169
76,nigth_club,37.430002,-5.974612
77,nigth_club,37.406172,-5.930830
78,nigth_club,37.401264,-5.925258


In [203]:
markers = {
    10051: ["stadium" , "orange", "black", "baseball"],
    16033: ["dog_park", "green", "white", "dog"],
    13377: ["vegan_rest", "white", "green", "leaf"],
    10032: ["nigth_club", "black", "white", "champagne-glasses"]
}

In [276]:
df = city_places("Valencia")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        27 non-null     object 
 1   place_name  27 non-null     object 
 2   lable       27 non-null     int64  
 3   lat         27 non-null     float64
 4   lon         27 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ KB


In [299]:
vlc_map = Map(locations["Valencia"], zoom_start=12)
df = city_places("Valencia")

In [303]:
### Subset the dataframe per layer

stadium = df[df["lable"] == 10051] # subset
dog_park = df[df["lable"] == 16033]
vegan_rest = df[df["lable"] == 13377]
nigth_club = df[df["lable"] == 10032]

In [304]:
stadium_group = folium.FeatureGroup(name=f"Sports_Stadiums: {stadium.shape[0]}")
HeatMap(data = stadium[["lat", "lon"]]).add_to(stadium_group)
stadium_group.add_to(vlc_map)

In [306]:
#vlc_map

In [307]:
#vlc_map